# Voxel pipeline

### Import requirements

In [1]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np

from ImageProcessor import ImageProcessor
from StereoMatcher import StereoMatcher
from VoxelGrid import VoxelGrid

%matplotlib ipympl

### Load calibrations and other data

In [2]:
imageProcessor = ImageProcessor()
imageProcessor.verbose = True
imageProcessor.loadMonoCalibrationResults()
imageProcessor.loadStereoCalibration()
imageProcessor.loadCameraProperties()
imageProcessor.loadStereoRectify()

Reading from data/monoCalibration.json
Loaded mono calibration
Reading from data/stereoCalibration.json
Loaded stereo calibration
Reading from data/cameraProperties.json
Loaded camera properties
Reading from data/stereoRectify.json
Loaded stereo rectification data


### Create stereo matcher

In [3]:
stereoMatcher = StereoMatcher(imageProcessor=imageProcessor, \
                matcher="SGBM", vertical=True, createRightMatcher=False)

imageProcessor.initUndistortRectifyMap()
#stereoMatcher.createDisparityWLSFilter()

Reading from data/parametersSGBM.json


### Loading images

In [4]:
path = "testImages/voxelTestImages"
imageGlobL = sorted(glob.glob("".join([path, "/top_*", ".png"])))
imageGlobR = sorted(glob.glob("".join([path, "/bottom_*", ".png"])))
print ("Selections: 0-{}".format(len(imageGlobL)-1))

Selections: 0-0


### Select image pair and display

In [5]:
imageNumber = 0

imageL = cv2.imread(imageGlobL[imageNumber])
imageR = cv2.imread(imageGlobR[imageNumber])

plt.figure()
plt.imshow(cv2.cvtColor(np.hstack([imageL, imageL]), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Convert to grayscale and undistort

In [6]:
imageProcessor.convertToGrayscale(imageL, imageR)
imageProcessor.undistortRectifyRemap(imageProcessor.grayImageL, \
                                        imageProcessor.grayImageR)

### View undistorted image

In [7]:
fig = plt.figure()
fig.suptitle("left/right undistorted")
plt.imshow(cv2.cvtColor(np.hstack([imageProcessor.undistortImageL, \
            imageProcessor.undistortImageR]), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
fig = plt.figure()
fig.suptitle("horizontal epipolar")
plt.imshow(cv2.cvtColor(imageProcessor.drawHorEpipolarLines(\
        imageProcessor.undistortImageL, imageProcessor.undistortImageR), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
fig = plt.figure(figsize=(6,15))
fig.suptitle("left/right undistorted")
plt.imshow(cv2.cvtColor(imageProcessor.drawVertEpipolarLines(\
        imageProcessor.undistortImageL, imageProcessor.undistortImageR), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute disparity map

In [10]:
stereoMatcher.computeDisparity(\
                grayImageL=imageProcessor.undistortImageL, \
                grayImageR=imageProcessor.undistortImageR)

stereoMatcher.clampDisparity()
stereoMatcher.applyClosingFilter()
#stereoMatcher.applyWLSFilterDisparity()

print ("minDisparity:", stereoMatcher.disparityMapL.min())
print ("maxDisparity:", stereoMatcher.disparityMapL.max())

minDisparity: 9.0
maxDisparity: 41.0


### View disparity map

In [11]:
plt.figure()
plt.imshow(cv2.rotate(stereoMatcher.disparityMapL, \
    cv2.ROTATE_90_CLOCKWISE))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute depth map

In [12]:
focalLength = imageProcessor.projectionMatrixL[0][0] # changes with rectify?
baseline = 32 # mm, measured irl

stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==0] = 0.9
stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==-1] = 0.9

depthMap = np.empty_like(stereoMatcher.disparityMapL)
depthMap = (focalLength*baseline)/stereoMatcher.disparityMapL[:]

print (stereoMatcher.disparityMapL.min())
print (stereoMatcher.disparityMapL.max())
print (depthMap.shape)
print (depthMap.min())
print (depthMap.max())

9.0
41.0
(640, 360)
481.33795
2192.7617


### View depth map

In [13]:
plt.figure()
plt.imshow(cv2.rotate(depthMap, cv2.ROTATE_90_CLOCKWISE))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# Use to save images
# _=cv2.imwrite("depth.png", cv2.rotate(depthMap, cv2.ROTATE_90_CLOCKWISE))

### Compute point cloud

In [15]:
voxelGrid = VoxelGrid(stereoMatcher, imageProcessor)
voxelGrid.verbose = True

In [16]:
voxelGrid.generatePointCloud()

Points in unfiltered pointcloud: 11520; completed in 0.00217 sec


Filtering extreme points

In [17]:
voxelGrid.filterPointCloud()

Points in filtered pointcloud: 9245; completed in 0.00092 sec


Rotate point cloud to have y forward

In [18]:
voxelGrid.redefinePointCloudCoordinate()

### View point cloud

In [19]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection = "3d")

ax.scatter(voxelGrid.pointCloud[:,0], voxelGrid.pointCloud[:,1], \
    voxelGrid.pointCloud[:,2], s=1)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

# Camera axis begins at x=0 and looks to positive x
ax.set_xlim(0,2000)
ax.set_ylim(-1000,1000)
ax.set_zlim(-1000,1000)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Voxelize point cloud

In [20]:
voxelGrid.resetVoxelGrid()
voxelGrid.voxelizePointCloud()

Voxel grid reset
Voxels in grid: 62; completed in 0.00932 sec; 67 iterations


### View voxelized point cloud

In [21]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection = "3d")

ax.scatter(voxelGrid.voxelGrid[:,0], \
        voxelGrid.voxelGrid[:,1], \
        voxelGrid.voxelGrid[:,2])

ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

# Camera axis begins at x=0 and looks to positive x
ax.set_xlim(0,2000)
ax.set_ylim(-1000,1000)
ax.set_zlim(-1000,1000)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Refine voxel grid with newer data

Interchanging data from calibration since calibration was 
done vertically

The x y notations are in image coordinates

In [22]:
# Horizontal field of view (degrees)
fovX = (imageProcessor.fovYL+imageProcessor.fovYR)/2
print("Horizontal:" fovX)
# Vertical field of view (degrees)
fovY = (imageProcessor.fovXL+imageProcessor.fovXR)/2
print(fovY)

56.95070190641179
33.98179171348929


### Checking for voxels in range

The refinement is done by simply replacing the older voxels, in view of the camera in the base grid, with the new voxels.

Here the voxels that may potentially be affected are found.

In [23]:
# Do this after compensations for atleast camera translation have been made
# Performed on the base voxel grid
# Distances to every voxel in base grid from camera
cameraPosition = np.zeros([3])
distanceToVoxels = np.linalg.norm(voxelGrid.voxelGrid-cameraPosition, axis=1)
print(distanceToVoxels.shape)

# Distance within which points may be modified
distanceToCheck = 1500

# These are the points to check
# Performed on the base voxel grid
voxelsWithinDist = voxelGrid.voxelGrid[distanceToVoxels<=distanceToCheck]
print(voxelGrid.voxelGrid.shape)
print(voxelsWithinDist.shape)

# Checking bounds on the coordinate axes
# Performed on the base voxel grid
voxelCheckBound = [[voxelsWithinDist[:,0].min(), voxelsWithinDist[:,1].min(), \
                        voxelsWithinDist[:,2].min()], \
                    [voxelsWithinDist[:,0].max(), voxelsWithinDist[:,1].max(), \
                        voxelsWithinDist[:,2].max()]]
print(voxelCheckBound) # min, max

(62,)
(62, 3)
(62, 3)
[[550, -650, -250], [1250, 250, 250]]


### Angle ranges of rotated and translated (new) voxel grid

Could potentially be found from the rotation matrix and be offset 
by the camera fovs.

Computing yaws on the new voxel grid; xy plane. A line along x axis has 0 degrees of yaw.

Normally this would be the results of the latest iteration after 
they are rotated and translated.

Taking x forward

In [24]:
newVoxelGrid = voxelGrid.voxelGrid
yawToNewVoxels = np.arctan2(newVoxelGrid[:,1], newVoxelGrid[:,0]) # y, x
print(yawToNewVoxels.shape)

yawCheckBound = np.array([yawToNewVoxels.min(), yawToNewVoxels.max()])
print(yawCheckBound*180/np.pi)
print((yawCheckBound[1]-yawCheckBound[0])*180/np.pi)

print(yawToNewVoxels*180/np.pi)

(62,)
[-27.645975  24.443953]
52.08993227110366
[ -3.814075   21.03751     3.814075  -25.559963  -25.559963   11.309932
  11.309932   -3.3664606 -11.309932    3.814075   -8.972626  -14.743562
   3.814075  -24.443953   21.03751   -21.37062    11.309932   24.443953
  12.9946165  -3.814075   -3.814075  -20.22486   -20.22486   -20.22486
 -14.743562  -23.198591   -3.3664606  24.443953  -14.743562  -23.198591
   3.814075  -20.22486    -8.972626   12.9946165 -18.434948  -11.309932
  21.03751    12.9946165  -3.3664606  11.309932  -18.434948  -14.743562
  -8.972626   -3.814075  -16.38954    24.443953  -14.743562   21.03751
 -27.47443   -24.443953   24.443953  -18.434948  -23.749493  -23.749493
 -13.392497  -27.645975  -18.434948  -18.434948  -21.37062     3.3664606
 -10.007979  -11.309932 ]


Computing pitches on the new voxel grid; xz plane.

Taking x forward

In [25]:
pitchToNewVoxels = np.arctan2(newVoxelGrid[:,2], newVoxelGrid[:,0])
print(pitchToNewVoxels.shape)

pitchCheckBound = np.array([pitchToNewVoxels.min(), pitchToNewVoxels.max()])
print(pitchCheckBound*180/np.pi)
print((pitchCheckBound[1]-pitchCheckBound[0])*180/np.pi)

print(pitchToNewVoxels*180/np.pi)

(62,)
[-15.255118  15.255118]
30.510236456393518
[ -3.814075    4.3987055  11.309932   -2.489553  -12.264773    3.814075
  -3.814075    3.3664606 -11.309932   -3.814075    8.972626   -3.0127873
 -11.309932    5.1944284  12.9946165  -2.489553   11.309932    5.1944284
  -4.3987055  11.309932  -11.309932  -14.743562   -3.0127873  -8.972626
   8.972626   -2.726311   10.007979   -5.1944284  -8.972626  -13.392497
   3.814075    8.972626   -3.0127873 -12.9946165   2.726311    3.814075
  -4.3987055  12.9946165  -3.3664606 -11.309932    8.130102   14.743562
   3.0127873   3.814075  -10.007979  -15.255118    3.0127873 -12.9946165
  11.309932   -5.1944284  15.255118   -8.130102   11.309932    2.29061
   2.726311  -13.392497   -2.726311   13.392497    2.489553   10.007979
  10.007979   -3.814075 ]


Computing rolls on the new voxel grid; yz plane.

Taking x forward

In [27]:
rollToNewVoxels = np.arctan2(newVoxelGrid[:,2], newVoxelGrid[:,1])
print(rollToNewVoxels.shape)

rollCheckBound = np.array([rollToNewVoxels.min(), rollToNewVoxels.max()])
print(rollCheckBound*180/np.pi)
print((rollCheckBound[1]-rollCheckBound[0])*180/np.pi)

print(rollToNewVoxels*180/np.pi)

(62,)
[-174.80557  174.80557]
349.61115568653986
[-135.         11.309932   71.56505  -174.80557  -155.55605    18.434948
  -18.434948  135.       -135.        -45.        135.       -168.69008
  -71.56505   168.69008    30.963757 -173.65982    45.         11.309932
  -18.434948  108.43495  -108.43495  -144.46233  -171.86989  -156.8014
  149.03624  -173.65982   108.43495   -11.309932 -149.03624  -150.94539
   45.        156.8014   -161.56505   -45.        171.86989   161.56505
  -11.309932   45.       -135.        -45.        156.8014    135.
  161.56505   135.       -149.03624   -30.963757  168.69008   -30.963757
  158.96248  -168.69008    30.963757 -156.8014    155.55605   174.80557
  168.69008  -155.55605  -171.86989   144.46233   173.65982    71.56505
  135.       -161.56505 ]


From the results above, checking for pitch and roll will return 
inconclusive results unless the rotation is in an exact side profile of the
grid.

This can be mitigated by doing the check before rotating or translating, but that would require the base grid to be rotated as a whole, refined, and then re-rotated back.

To save on computation, the yaw range for the new voxel grid may simply be taken from camera properties already found during calibration. 

Then, to check where the camera is facing, a vector of unit distance from the camera facing along its axis could be rotated and translated along with the camera. 

Its yaw with respect to the camera center can then be calculated, and then offset by half its horizontal fov on both sides, to find the yaw limits within which the base voxels may be modified.